In [58]:
# algorithmic building blocks

def generate_prime(sizep):
    p = 0
    while not is_prime(p) or p % 4 != 3: 
    # p is congruent to 3 ensures we could use 1728, and N is congruent to 3 ensures R/NR is a field
        p = random_prime(2^sizep)
    return p

p = generate_prime(100) # p is of input size bits 
N = generate_prime(120) # N is of input size bits

K.<a> = NumberField(x^2 + 1)
R = K.ring_of_integers()
F.<aa> = R.ideal(N).residue_field()
red = F.reduction_map()
L = F.lift_map()
Q.<i,j,k> = QuaternionAlgebra(QQ,-1,-p)
ps = floor((log(p))^4)

def powersmooth_integer(S): 
# generates an integer that's ps-powersmooth and bigger than S
    pp = 2
    pd = 1
    while pd < S:
        e = floor(log(ps,pp))
        e = e if e%2 == 0 else e-1
        pd = pd*pp^e
        pp = pp.next_prime()
    return pd

def RepresentInteger_(N,A,B):
    S = floor(p*(log(p))^3)
    M = powersmooth_integer(S)
    m = floor(sqrt(S/p))
    for _ in range(m**2):
        z = randint(-m, m)
        t = randint(-m, m)
        M_ = M - p*(t^2+z^2)
        if gcd(z^2 + t^2,N) == 1 and gcd(M_,N) == 1 and M_.is_prime()==True and M_%4 == 1:
            (x,y) = sum_of_k_squares(2,M_)
            C = x+y*a
            D = z+t*a
            temp = A*B.conjugate()*C*D.conjugate()
            im = R(temp).imag()
            sq_test = 4*(4*p^2*(A*B*C*D).norm()-((A*C).norm()-p*(A*D).norm())^2)
            if kronecker_symbol(sq_test, N) == 1:
                break
    return C,D

def QuaternionDecomposition(N,A,B,C,D):
    A=R(A)
    B=R(B)
    C=R(C)
    D=R(D)
    #define the quadratic equation of x3 = s+ti, get x3
    f0 = A.norm()*(C.norm()-p*D.norm()) + 2*p*(A*B.conjugate()*C*D.conjugate())[0]
    f1 = -4*p*(A*B.conjugate()*C*D.conjugate())[1]
    f2 = A.norm()*(C.norm()-p*D.norm()) - 2*p*(A*B.conjugate()*C*D.conjugate())[0]
    ZN = Zmod(N)
    ZNx.<t> = PolynomialRing(ZN)
    f = ZN(f0)+ZN(f1)*t+ZN(f2)*t^2 #forcing s to be 1, could be anything nonzero, the equation for s,t is f0s^2+f1st + f2t^2
    x3 = red(1+ZZ(f.roots()[0][0])*a)
    #solve linear equation in F, in the notes, x,y are used to denote the solutions
    temp = (x3.norm()*red(C.conjugate())*red(D.conjugate())*red(p^3)*red(C.norm()+p*D.norm())).inverse()
    xx = temp*(red(p^2)*red(A*(D.conjugate())*(D.conjugate()))*x3 - red(p^2)*red(C.conjugate()*D.conjugate()*B)*x3.conjugate())
    yy = temp*(red(p)*red(C*C.conjugate()*A)*x3+red(p^2)*red(C.conjugate()*D*B)*x3.conjugate())
    #find x1 and x2 such that x1*x2 = y and x1*x2bar = x
    u1 = xx[0]
    v1 = xx[1]
    u2 = yy[0]
    v2 = yy[1]
    M1 = (u1+u2)/2
    M2 = (u1-u2)/2
    M3 = (v1+v2)/2
    M4 = (v2-v1)/2
    s1 = 1
    t1 = M1*s1/M3
    s2 = M2/s1
    t2 = M1/t1
    x1 = (t1+s1*aa)
    x2 = t2+s2*aa
    #lifting x1,x2,x3 to R
    x11 = L(x1)
    x22 = L(x2)
    x33 = L(x3)
    #construct a quaternion algebra
    Q.<i,j,k> = QuaternionAlgebra(QQ,-1,-p)
    sigma0 = A[0]+A[1]*i+(B[0]+B[1]*i)*j
    gamma = C[0]+C[1]*i+(D[0]+D[1]*i)*j
    alpha1 = (x11[0]+x11[1]*i)*j
    alpha2 = (x22[0]+x22[1]*i)*j
    alpha3 = (x33[0]+x33[1]*i)*j
    return alpha1,alpha2,alpha3

def find_small_prime(N):
    pp = 2
    for _ in range(N):
        #print(_)
        if kronecker_symbol(pp, N) == -1:
            return pp
            break
        else:
            pp = next_prime(pp)

def solve_linear(a, b, c, N):
    # Generate a random x
    x = mod(randint(0, N-1),N)

    # Compute y = (c - ax) / b mod N
    y = Mod((c - a * x) / b, N)
    return x, y

def StrongApproximation_ps(N,mu0): #mu0 in Rj, (n(mu0),N)=1
    S = p*N^4
    t = Q((mu0*j)/(-p))[0]
    s = Q((mu0*j)/(-p))[1]
    if kronecker_symbol(p*(t^2+s^2), N) == 1:
        F = powersmooth_integer(S)
    else:
        F = powersmooth_integer(S)*find_small_prime(N)
    lam = ZZ((mod(F,N)/mod(p*(s^2+t^2),N)).sqrt())
    temp = (ZZ(F-lam^2*p*(t^2+s^2)))/N
    coex = mod(lam*p*2*t,N)
    coey = mod(lam*p*2*s,N)
    cont = mod(temp,N)
    for _ in range(N):
        c,d = solve_linear(coex, coey, cont, N)
        temp1 = ZZ((F-p*((ZZ(lam)*ZZ(t) + ZZ(c)*N)^2+(ZZ(lam)*ZZ(s) + ZZ(d)*N)^2)))
        M = temp1/(N^2)
        if ZZ(M).is_prime()==True and ZZ(M)%4 == 1:
            (a,b) = sum_of_k_squares(2,M)
            break    
    return lam*mu0+N*((ZZ(a)+ZZ(b)*i)+(ZZ(c)+ZZ(d)*i)*j),lam

In [59]:
# given sigma0 in R+Rj such that n(sigma0,N)=1, solve the PQLP for sigma0 when N is a prime and O_0 is chosen to be isomorphic to End(E_1728)
# we generate random p,N in the cell above with p congruent to 3 mod 4 to ensure E_1728 is supersingular, and N congruent to 3 mod 4 ensures R/NR is a field

def PQLP_O0(N,sigma0): # returns sigma and lambda such that n(sigma) is powersmooth and sigma = lambda*sigma0 mod NO_0
    A = R(sigma0[0]+sigma0[1]*a)
    B = R(sigma0[2]+sigma0[3]*a)
    C,D = RepresentInteger_(N,A,B)
    alpha1,alpha2,alpha3 = QuaternionDecomposition(N,A,B,C,D)
    gamma1,lambda1 = StrongApproximation_ps(N,alpha1)
    #print((gamma1-lambda1*alpha1)/N)
    gamma2,lambda2 = StrongApproximation_ps(N,alpha2)
    #print(gamma2)
    gamma3,lambda3 = StrongApproximation_ps(N,alpha3)
    #print(gamma3)
    gamma = C[0]+C[1]*i+(D[0]+D[1]*i)*j
    return gamma1*gamma*gamma2*gamma*gamma3, lambda1*lambda2*lambda3

def verifyPQLP(sigma0,N): # verifies whether sigma = lambda*sigma0 mod NO_0 and compute the factors of the norm of sigma
    sigma,lambdaa = PQLP_O0(N,sigma0)
    difference = (sigma-lambdaa*(sigma0))
    return difference[0]%N == 0 and difference[1]%N == 0 and difference[2]%N == 0 and difference[3]%N == 0 and gcd(lambdaa,N) == 1,factor(sigma[0]^2+sigma[1]^2+p*(sigma[2]^2+sigma[3]^2))


In [60]:
sigma0 = randint(2^100,2^101)+randint(2^100,2^101)*i+(randint(2^100,2^101)+randint(2^100,2^101)*i)*k # define a random sigma0
PQLP_O0(N,sigma0)

(438651374275684106757804520773557670787984911480124106017343494798113649540357741808212383191110717781959382802646134594624726525087060085729866621160565507170831251684953495422422569926903241090558586580971079003951466344854722940516782861325011390961606649047166197585727857075294332379459140678894400 - 283383953621514230094943805960252228909907611281114930669690619945400278871525841815435620114713263263015077430431801983320799804449836722194879496191601690296672753955765144065237532298837096747866292880572968865742225464282002820274464900180418021752270090133765843575306384150965692886975722494838400*i + 7046069938812263196953319605870383130324989444844158675483851384378356628357604399420870527995293727771023500377336981076571470593766396842413161580498763346109144802611912143187403904516631974684566277797508886959712728369306130203746803094052504746385967790502612311329724693588212800*j + 2350671420473075348683369993296548662319791727276573816653377222867556685264271048845961311373

In [61]:
verifyPQLP(sigma0,N)

(True,
 2^120 * 3^70 * 5^50 * 7^41 * 11^30 * 13^30 * 17^12 * 19^12 * 23^12 * 29^12 * 31^12 * 37^12 * 41^12 * 43^12 * 47^12 * 53^12 * 59^12 * 61^12 * 67^12 * 71^6 * 73^6 * 79^6 * 83^6 * 89^6 * 97^6 * 101^6 * 103^6 * 107^6 * 109^6 * 113^6 * 127^6 * 131^6)